### Generating Events

In [1]:
import numpy as np

# Event types
floatevent = 0

# Generate event of random size within range
# Return array representing event
def generate_event(event_size, rand_multiplier = 10):
    return np.array(rand_multiplier*np.random.rand(1, event_size), dtype='int16')

# Return array of n events of form (time, data)
# with times separated within dt range
# with data within size range
def generate_n_events(n, dt_min, dt_max, event_size):
    t = 0
    events = []
    for i in range(n):
        events.append((t, generate_event(event_size)))
        dt = dt_min + (np.random.random() * (dt_max - dt_min))
        t += dt
    return events

generate_n_events(3, 0.5, 0.8, 5000)

[(0, array([[0, 8, 2, ..., 9, 4, 5]], dtype=int16)),
 (0.7477541406208948, array([[4, 8, 6, ..., 4, 7, 2]], dtype=int16)),
 (1.2635961136026073, array([[9, 6, 8, ..., 1, 1, 4]], dtype=int16))]

### Simulation

In [6]:
import time
import h5py

# Run a simulation with the given parameters
# n_events is number of events to group together to write to hdf5
def run_sim(n, group_size, dt_min, dt_max, event_size, file="out.h5"):
    hf = h5py.File('data.h5', 'w')
    
    events = generate_n_events(n, dt_min, dt_max, event_size)
    
    group = []
    written = 0
    count = 0
    start = time.time()
    # Write the events to hdf5
    while count < n:
        now = time.time() - start

        # Next event has not fired yet
        if events[count][0] > now:
            time.sleep(0.01)
            continue
        
        # If the event has fired
        if group_size == 1:
            hf.create_dataset("event%d"%(count), data=events[count][1][0])
            written += 1
            count += 1
        elif group_size > 1:
            group.append(events[count][1][0])
            if len(group) >= group_size or count == n-1:
                hf.create_dataset("events%d"%(written), data=group)
                written += 1
                group = []
            count += 1

    hf.close()
    
    return time.time() - start

### Confirming data from hdf5

In [7]:
n_events = 5
print(run_sim(n_events, 2, 1, 2, 100), "s")

5.369687080383301 s


In [19]:
f = h5py.File('data.h5', 'r')
keys = list(f.keys())
print("Keys:", keys, "\n")

count = 0
for key in keys:
    data = np.array(f[key])
    print("Group %d size:"%count, data.shape)
    count += 1

print("First group:", list(f[key]))

f.close()
# Get the data

Keys: ['events0', 'events1', 'events2'] 

Group 0 size: (2, 100)
Group 1 size: (2, 100)
Group 2 size: (1, 100)
First group: [array([6, 2, 6, 0, 0, 6, 0, 9, 0, 1, 9, 1, 3, 7, 7, 6, 4, 1, 5, 9, 3, 0,
       3, 5, 7, 8, 9, 8, 7, 8, 3, 4, 4, 4, 9, 5, 9, 3, 4, 5, 8, 8, 0, 1,
       3, 3, 4, 1, 0, 9, 6, 2, 2, 4, 0, 9, 1, 7, 7, 7, 7, 9, 6, 0, 8, 7,
       9, 8, 8, 3, 0, 6, 7, 7, 9, 5, 5, 4, 0, 5, 7, 7, 6, 9, 4, 8, 6, 1,
       4, 6, 9, 2, 1, 7, 5, 2, 3, 8, 0, 2], dtype=int16)]


### Testing